In [ ]:
pip install netcdf4

     |████████████████████████████████| 4.7MB 6.0MB/s 
     |████████████████████████████████| 317kB 42.1MB/s 


In [ ]:
pip install geopandas

     |████████████████████████████████| 1.0MB 4.4MB/s 
     |████████████████████████████████| 15.3MB 229kB/s 
     |████████████████████████████████| 6.5MB 37.1MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import netCDF4 as nc
import numpy as np
import seaborn as sns
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import shapely
import matplotlib.pyplot as plt
import os
import os.path
from os import path
from datetime import datetime
from tqdm.auto import tqdm

In [ ]:
os.chdir('/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/IMERG/')

In [ ]:
os.getcwd()

'/content/gdrive/.shortcut-targets-by-id/1QPqq4-DIl88pfOZGc2mdgGZONJn6bIid/rainfall-forecast/precipitation_data/IMERG'

In [ ]:
dataset1 = '3B-DAY-L.MS.MRG.3IMERG.20160625-S000000-E235959.V06.nc4'
dataset1 = nc.Dataset(dataset1)

dataset2 = '3B-DAY-L.MS.MRG.3IMERG.20200828-S000000-E235959.V06.nc4'
dataset2 = nc.Dataset(dataset2)


In [ ]:
dataset1

In [ ]:
lats = dataset1.variables['lat'][:]
lons = dataset1.variables['lon'][:]
lat_bnds, lon_bnds = [-11.183, 3.622], [28.417, 41.694]

lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
lat_inds = np.asarray(lat_inds)

lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))
lon_inds = np.asarray(lon_inds)

dataset1_subset = dataset1.variables['HQprecipitation'][:, lat_inds[0, 0]:lat_inds[0, -1], lon_inds[0, 0] : lon_inds[0, -1]]
dataset1_subset = dataset1_subset.filled(np.nan)

dataset2_subset = dataset2.variables['HQprecipitation'][:, lat_inds[0, 0]:lat_inds[0, -1], lon_inds[0, 0] : lon_inds[0, -1]]
dataset2_subset = dataset2_subset.filled(np.nan)

In [ ]:
lat_inds.shape


(1, 148)

In [ ]:
data_lat_subset = dataset1.variables['lat'][lat_inds[0, 0]:lat_inds[0, -1]]
data_lat_subset = data_lat_subset.filled(np.nan)

data_lon_subset = dataset1.variables['lon'][lon_inds[0, 0] : lon_inds[0, -1]]
data_lon_subset = data_lon_subset.filled(np.nan)

In [ ]:
date_range = pd.date_range('01/01/2016', '10/30/2020') #.tolist()

# make list of filenames with years 2016-2020 
filelist = []
for date in date_range:
  year = date.date().strftime('%Y')
  string = '3B-DAY-L.MS.MRG.3IMERG.' + year
  filelist.append(string)

months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
print(filelist)

['3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY-L.MS.MRG.3IMERG.2016', '3B-DAY

In [ ]:
# try out merge loop
dataset_list = []
for day in range(1,36):
  day = str(day).zfill(2)
  file_string = '/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/IMERG/3B-DAY-L.MS.MRG.3IMERG.201705' + day + '-S000000-E235959.V06.nc4'
  if os.path.exists(file_string):
    dataset = nc.Dataset(file_string)
    print(dataset)
    dataset_subset = dataset.variables['HQprecipitation'][:, lon_inds[0, 0] : lon_inds[0, -1], lat_inds[0, 0]:lat_inds[0, -1]]
    dataset_subset = dataset_subset.filled(np.nan)
    dataset_subset_early = dataset_subset[0, :, :]
    #dataset_subset_late = dataset_subset[1, :, :] # @robert can we skip this one? time has only one dimension (without it it works)
    dataset_list.append(dataset_subset_early)
    #dataset_list.append(dataset_subset_late)
    
monthly_merged_data = np.sum(dataset_list, axis = 0)
print(monthly_merged_data.shape)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    BeginDate: 2017-05-01
    BeginTime: 00:00:00.000Z
    EndDate: 2017-05-01
    EndTime: 23:59:59.999Z
    FileHeader: StartGranuleDateTime=2017-05-01T00:00:00.000Z;
StopGranuleDateTime=2017-05-01T23:59:59.999Z
    InputPointer: 3B-HHR-L.MS.MRG.3IMERG.20170501-S000000-E002959.0000.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S003000-E005959.0030.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S010000-E012959.0060.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S013000-E015959.0090.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S020000-E022959.0120.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S023000-E025959.0150.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S030000-E032959.0180.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S033000-E035959.0210.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S040000-E042959.0240.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S043000-E045959.0270.V06B.HDF5;3B-HHR-L.MS.MRG.3IMERG.20170501-S050000-E052959.0300

In [ ]:
# loop merging the data
for file1 in tqdm(filelist):
  for month in months:
    dataset_list = []
    for day in range(1,33):
      day = str(day).zfill(2)
      file_string = '/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/IMERG/' + file1 + month + day + '-S000000-E235959.V06.nc4'
      if os.path.exists(file_string):
        dataset = nc.Dataset(file_string)
        dataset_subset = dataset.variables['HQprecipitation'][:, lon_inds[0, 0] : lon_inds[0, -1], lat_inds[0, 0]:lat_inds[0, -1]]
        dataset_subset = dataset_subset.filled(np.nan)
        dataset_subset_early = dataset_subset[0, :, :]
        #dataset_subset_late = dataset_subset[1, :, :]
        dataset_list.append(dataset_subset_early)
        #dataset_list.append(dataset_subset_late)
    
  monthly_merged_data = np.sum(dataset_list, axis = 0)
  #print(daily_merged_data.shape)
  #plt.imshow(daily_merged_data)
  
  dataset_name = '/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/IMERG/IMERG_monthly/' + file1 + month + '.nc'
  
  dataset_join = nc.Dataset(dataset_name, 'w', format = 'NETCDF4')

  dataset_join.createDimension('lon', data_lon_subset.shape[0])
  dataset_join.createDimension('lat', data_lat_subset.shape[0])
  dataset_join.createDimension('time', None)

  longitude = dataset_join.createVariable('Longitude', 'f4', 'lon')
  latitude = dataset_join.createVariable('Latitude', 'f4', 'lat')  
  precip = dataset_join.createVariable('precip', 'f4', ('time', 'lat', 'lon'))
  time = dataset_join.createVariable('time', 'i4', 'time')

  longitude[:] = data_lon_subset
  latitude[:] = data_lat_subset
  precip[0, :, :] = monthly_merged_data
  
  dataset_join.close()
  #print('check')